<a href="https://colab.research.google.com/github/nanopiero/fusion/blob/main/notebooks/construction_pb_fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Construction problème fusion


In [1]:
# Imports des bibliothèques utiles
# pour l'IA
import torch
# pour les maths
import numpy as np
# pour afficher des images et des courbes
import matplotlib.pyplot as plt

from random import randint
import os

In [2]:
! git clone https://github.com/nanopiero/fusion.git

Cloning into 'fusion'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (75/75), done.
Receiving objects: 100% (76/76), 992.12 KiB | 11.40 MiB/s, done.
remote: Total 76 (delta 42), reused 4 (delta 0), pack-reused 0
Resolving deltas: 100% (42/42), done.


In [3]:
# Import des fonctions génératrices exploitées à l'échelle de l'image
os.chdir('fusion')
import utile_fusion
# import importlib
# importlib.reload(utile_fusion)
from utile_fusion import spatialized_gt, create_cmls_filter



In [7]:
# Exemple :
S = 64
# in the dataset :
image = spatialized_gt(ndiscs=5, size=S, nsteps=60)
pairs, filter = create_cmls_filter(S, npairs = 10)

In [8]:
# Import des fonctions utilisées à l'échelle du batch, sur carte GPU
from utile_fusion import point_gt, segment_gt, make_noisy_images

In [9]:
# Exemple
device = torch.device('cuda:0')
images = torch.tensor(image).unsqueeze(0).float().to(device)
pairs = torch.tensor(pairs).unsqueeze(0).float().to(device)
filters = torch.tensor(filter).unsqueeze(0).float().to(device)

# done on gpu:
point_measurements = point_gt(images, npoints=5)
segment_measurements = segment_gt(images, pairs, filters)


print(pairs.min(), pairs.max())
images2 = make_noisy_images(images)

tensor(0.0067, device='cuda:0') tensor(0.9644, device='cuda:0')
shape of gk: torch.Size([1, 1, 25, 25])
tensor([0.2853], device='cuda:0')


In [10]:
#Import des fonctions de visualisation
from utile_fusion import set_tensor_values2, plot_images

In [ ]:
# Exemple / tracés
images = set_tensor_values2(images, point_measurements)
plot_images(images.cpu().numpy().squeeze(0) + filter.sum(axis=0),
            images2.squeeze(0).cpu().numpy(),
            point_measurements.cpu().numpy().squeeze(0),
            segment_measurements.cpu().numpy().squeeze(0))